# Tabular Data Processing with TabuLa-8B

This notebook demonstrates how to use TabuLa-8B for zero-shot inference on tabular data. For best performance, this notebook should be run with access to a GPU.

TabuLa-8B supports:
- Zero-shot and few-shot inference
- Both categorical and continuous inputs
- Flexible input format using pandas DataFrames

**Note**: This notebook requires significant computational resources due to the model size.

## Setup

First, let's install the required packages. After running this cell, you'll need to restart the runtime.

In [1]:
%%capture
!pip install --upgrade pip

# Install core dependencies
!pip install torch transformers accelerate bitsandbytes
!pip install pandas numpy nltk

# Install rtfm and related packages
!pip install git+https://github.com/mlfoundations/rtfm.git
!pip install git+https://github.com/mlfoundations/tabliblib.git

print("⚠️ Please restart the runtime now using Runtime → Restart runtime")

## Model Loading and Setup

After restarting the runtime, run this cell to load the model and tokenizer.

In [2]:
import os
import nltk
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from accelerate import init_empty_weights

# Download NLTK data
nltk.download('punkt')

# Model configuration
MODEL_NAME = "mlfoundations/tabula-8b"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {DEVICE}")

# Load configuration
config = AutoConfig.from_pretrained(MODEL_NAME)
config.torch_dtype = torch.bfloat16

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Load model with memory efficiency settings
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    config=config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    load_in_8bit=True if DEVICE == "cuda" else False
)

print("✅ Model loaded successfully")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

...

model-00006-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/2.57G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

✅ Model loaded successfully


## Inference Function

Let's create a function to handle tabular inference.

In [3]:
def format_tabular_prompt(row, target_column=None):
    """
    Format a pandas row into a text prompt.
    """
    prompt = "Table data:\n"
    for col, val in row.items():
        if col != target_column:
            prompt += f"{col}: {val}\n"
    if target_column:
        prompt += f"\nPredict {target_column}:"
    return prompt

def predict_tabular(model, tokenizer, data, target_column, max_length=100):
    """
    Make predictions on tabular data.
    """
    prompt = format_tabular_prompt(data.iloc[0], target_column)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    prediction = response[len(prompt):].strip()

    return prediction

## Example: Weather Prediction

Let's try a simple weather prediction example.

In [4]:
# Create a sample weather dataset
weather_data = pd.DataFrame([
    {
        "temperature": 25,
        "humidity": 60,
        "wind_speed": 10,
        "cloud_cover": "partial",
        "time_of_day": "afternoon"
    }
])

# Make prediction
prediction = predict_tabular(
    model,
    tokenizer,
    weather_data,
    target_column="weather_condition"
)

print("Input data:")
display(weather_data)
print(f"\nPredicted weather condition: {prediction}")

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Input data:


,temperature,humidity,wind_speed,cloud_cover,time_of_day
0,25,60,10,partial,afternoon



Predicted weather condition: sunny


## Memory Management

If you're done with predictions, you can free up memory:

In [5]:
# Free up memory
import gc
del model
del tokenizer
torch.cuda.empty_cache()
gc.collect()

print("✅ Memory cleared")

✅ Memory cleared


## Conclusion

This notebook demonstrated a simplified approach to using TabuLa-8B for tabular inference:

1. Memory-efficient model loading using 8-bit quantization
2. Simple prompt formatting for tabular data
3. Basic prediction functionality
4. Proper memory management

For more advanced features and better performance, consider:
- Using a machine with more GPU memory
- Implementing batch processing for multiple predictions
- Adding support for few-shot learning with examples
- Implementing proper error handling and validation